<a href="https://colab.research.google.com/github/Sugumaran18/CODSOFT-AI/blob/main/Recommendation%20System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, Add
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [3]:
# Load VGG16 model pretrained on ImageNet
vgg = VGG16(weights='imagenet')
vgg = Model(inputs=vgg.inputs, outputs=vgg.layers[-2].output)  # Remove the last layer

# Preprocess image for VGG16
def preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.vgg16.preprocess_input(img)
    return img

# Extract features using VGG16
def extract_features(image_path):
    img = preprocess_image(image_path)
    features = vgg.predict(img)
    return features

# Load image and extract features
image_path = 'path_to_image.jpg'
image_features = extract_features(image_path)

# Example captions and preprocessing
captions = ['a man riding a horse', 'a person on a horse', 'a man on a horse']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(captions)
vocab_size = len(tokenizer.word_index) + 1

# Convert captions to sequences
sequences = tokenizer.texts_to_sequences(captions)
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Build the image captioning model
embedding_dim = 256

# Image feature input
image_input = Input(shape=(4096,))
image_model = Dense(embedding_dim, activation='relu')(image_input)
image_model = RepeatVector(max_length)(image_model)

# Caption input
caption_input = Input(shape=(max_length,))
caption_model = Embedding(vocab_size, embedding_dim, input_length=max_length)(caption_input)
caption_model = LSTM(256, return_sequences=True)(caption_model)
caption_model = Dropout(0.5)(caption_model)

# Combine image features and caption input
combined = Add()([image_model, caption_model])
output = LSTM(256)(combined)
output = Dense(vocab_size, activation='softmax')(output)

# Final model
model = Model(inputs=[image_input, caption_input], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Model summary
model.summary()

# Training code (for illustration, not complete)
# Assume you have training data of image_features and corresponding captions

# For each epoch, you would pass the image features and corresponding captions to train the model.
# model.fit([image_features, padded_sequences], targets, epochs=20, batch_size=32)

# Inference Example
def generate_caption(image_features, tokenizer, max_length):
    input_sequence = [tokenizer.word_index['startseq']]
    for i in range(max_length):
        sequence = pad_sequences([input_sequence], maxlen=max_length, padding='post')
        yhat = model.predict([image_features, sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = tokenizer.index_word[yhat]
        input_sequence.append(yhat)
        if word == 'endseq':
            break
    return ' '.join([tokenizer.index_word[idx] for idx in input_sequence if idx > 0])

# Assuming image_features is extracted from the input image
caption = generate_caption(image_features, tokenizer, max_length)
print(f"Generated Caption: {caption}")

553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_image.jpg'